<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-mcp/blob/main/openai-mcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using OpenAI with a MCP Server

In [ ]:
# Install required packages
!pip install --upgrade pip
!pip install fastmcp openai nest_asyncio

In [ ]:
# Your weather_server.py code
from fastmcp import FastMCP
import nest_asyncio
import threading
import time

nest_asyncio.apply()

mcp = FastMCP(
    name="WeatherServer🚀",
    instructions="This provides an up to date weather forecast for any location."
)

# Tool 1: Forecast
@mcp.tool("get_forecast")
def get_forecast(location: str):
    return {"forecast": f"Sunny in {location}"}

# Tool 2: Alerts
@mcp.tool(name="get_alerts")
def get_alerts(location: str):
    return {"alerts": f"No severe alerts currently for {location}"}

# Tool 3: Math
@mcp.tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers together."""
    return a * b

# Resource 1: Climate Data
@mcp.resource(uri="http://example.com", name="climate_data")
def climate_data():
    """Return static climate information."""
    return {
        "Berlin": {"avg_temp": "10°C", "rainfall": "570mm"},
        "Boise": {"avg_temp": "12°C", "rainfall": "300mm"},
        "Tokyo": {"avg_temp": "16°C", "rainfall": "1500mm"}
    }

# Prompt 1: Analysis of Numerical data
@mcp.prompt
def analyze_data(data_points: list[float]) -> str:
    """Creates a prompt asking for analysis of numerical data."""
    formatted_data = ", ".join(str(point) for point in data_points)
    return f"Please analyze these data points: {formatted_data}"

# # Run server in background thread
# def run_server():
#     mcp.run(port=8000)

# server_thread = threading.Thread(target=run_server, daemon=True)
# server_thread.start()

# # Give the server time to start
# time.sleep(3)
# print("FastMCP server is running on port 8000")

# Define the function to run the server
def run_server():
  # Use transport="streamable-http" for compatibility with notebooks/Colab
  print("🚀 Starting FastMCP server in background thread...")
  mcp.run(transport="streamable-http", host="0.0.0.0", port=8000)

# Start the server in a separate thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give the server a moment to start up
time.sleep(2)
print("✅ Server should be running. Access it at http://localhost:8000/mcp")

In [ ]:
# Set your API key
client = OpenAI(api_key="your-api-key-here")

# Now you can use the OpenAI client
# The FastMCP server is running in the background on localhost:8000
# and can be accessed by your agent if configured to do so

# Example OpenAI Agent code
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": "What's the weather like?"}
    ]
)

print(response.choices[0].message.content)